##### 1. How many customers has Foodie-Fi ever had?

```sql 
SELECT COUNT(DISTINCT(customer_id)) AS customer_count 
FROM foodie_fi.subscriptions
```

<div>
<img src="../sql-screenshots/foodie_fi_images/s1.png" width="250"/>
</div>

##### 2. What is the monthly distribution of trial plan start_date values for our dataset - use the start of the month as the group by value

```sql 
SELECT DATE_TRUNC('month', start_date) AS start_date_trunc, 
COUNT(*) AS trial_count FROM foodie_fi.subscriptions
LEFT JOIN foodie_fi.plans 
ON subscriptions.plan_id = plans.plan_id 
WHERE subscriptions.plan_id = 0 
GROUP BY start_date_trunc
ORDER BY start_date_trunc
```

<div>
<img src="../sql-screenshots/foodie_fi_images/s2.png" width="250"/>
</div>

##### 3. What plan start_date values occur after the year 2020 for our dataset? Show the breakdown by count of events for each plan_name

```sql 
SELECT subscriptions.plan_id, plans.plan_name, 
COUNT(subscriptions.plan_id) AS plan_count
FROM foodie_fi.subscriptions
INNER JOIN foodie_fi.plans 
ON plans.plan_id = subscriptions.plan_id
WHERE EXTRACT('YEAR' FROM start_date) > 2020
GROUP BY subscriptions.plan_id, plans.plan_name
ORDER BY plan_count
```

<div>
<img src="../sql-screenshots/foodie_fi_images/s3.png" width="300"/>
</div>

##### 4. What is the customer count and percentage of customers who have churned rounded to 1 decimal place?

```sql 
SELECT
  SUM(CASE WHEN plan_id = 4 THEN 1 ELSE 0 END) AS churn_customers,
  ROUND(
    100 * SUM(CASE WHEN plan_id = 4 THEN 1 ELSE 0 END) /
      COUNT(DISTINCT customer_id)
  ,1) AS percentage
FROM foodie_fi.subscriptions;
```

<div>
<img src="../sql-screenshots/foodie_fi_images/s4.png" width="250"/>
</div>

##### 5. How many customers have churned straight after their initial free trial, what percentage is this rounded to the nearest whole number?

```sql 
WITH customer_table AS (
SELECT customer_id, subscriptions.plan_id, start_date, plan_name, 
  CASE WHEN plan_name = 'basic monthly' THEN start_date + 30 
       WHEN plan_name = 'trial' THEN start_date + 7 
       WHEN plan_name = 'pro annual' THEN start_date + 365 
       WHEN plan_name = 'pro monthly' THEN start_date + 30 
       ELSE start_date - 7 END AS end_date 
FROM foodie_fi.subscriptions
LEFT JOIN foodie_fi.plans 
ON subscriptions.plan_id = plans.plan_id
), period_table AS (
SELECT customer_id, plan_id,
  CASE WHEN (plan_id = 0) THEN STRING_AGG (
	start_date || ':' || end_date,
        ',')
    ELSE STRING_AGG (
	end_date || ':' || start_date,
        ',')
      END AS period
FROM customer_table
WHERE plan_id = 0 OR plan_id = 4 
GROUP BY customer_id, plan_id
ORDER BY customer_id, plan_id 
), churn_count AS ( -- contains the number of customers churned. 
  SELECT customer_id, COUNT(*) FROM period_table
GROUP BY customer_id, period
HAVING COUNT(*) > 1 
ORDER BY customer_id DESC
)

-- alternative solution
WITH customer_table AS (
SELECT customer_id, plan_id,  
ROW_NUMBER() OVER (PARTITION BY customer_id ORDER BY start_date) AS period_rank 
FROM foodie_fi.subscriptions
ORDER BY customer_id, plan_id
)

SELECT
  SUM(CASE WHEN plan_id = 4 THEN 1 ELSE 0 END) AS churn_customers,
  ROUND(
    100 * SUM(CASE WHEN plan_id = 4 THEN 1 ELSE 0 END) /
    COUNT(*)
  ) AS percentage
FROM customer_table
WHERE period_rank = 2;
```